<a href="https://colab.research.google.com/github/dcbfoss/MiniProjects/blob/main/Project1_Book_Suggestor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
import numpy as np
import pandas as pd

In [3]:
DATA_SOURCE = "https://github.com/dcbfoss/MiniProjects/raw/main/project1/data/books_usage_sample.xlsx"
RANGE = 5

In [5]:
def readfile(filename, sheetname):
    data=pd.read_excel(filename,sheet_name=[sheetname],header=None)
    return data

def predict_book(seed, model, books, booktitles):
    x = np.array(seed)
    x = np.reshape(x, (1,4))
    index = (np.argmax(model.predict(x, verbose=0)))
    bookname  = booktitles[books[index]]
    return (index, books[index], bookname)

In [39]:
usernames = []; books = []; booktitles = {}; usage_record = []
data = readfile(DATA_SOURCE, 'names')
usernames = [ i[0].lower() for i in data['names'].values]
usage_record = [[] for i in data['names'].values]

data = readfile(DATA_SOURCE, 'books')
books = [ i[0] for i in data['books'].values]
for key, value in data['books'].values:
    booktitles[key] = value

data = readfile(DATA_SOURCE, 'record')
for entry in data['record'].values:
    usage_record[usernames.index(entry[1].lower())].append(books.index(entry[3]))

max_embed = len(books)+2

# Statistics

In [54]:
weight_matrix = []; books_per_user = [len(books_) for books_ in usage_record]
for user in usernames:
    weight_matrix.append([0 for book in books])

for user_index in range(len(usernames)):
    importance_score = books_per_user[user_index] / sum(books_per_user)
    for book_index in range(len(books)):
        if book_index in usage_record[user_index]:count = usage_record[user_index].count(book_index)
        else:count = 0
        favorite_score = count/(books_per_user[user_index]/len(set(usage_record[user_index])))
        weight = favorite_score * importance_score
        weight_matrix[user_index][book_index] = weight

fav_summary = [0 for book in books]
for index, weights in enumerate(weight_matrix):
    for j, weight in enumerate(weights):
        fav_summary[j]+= weight
    max_weight_index = np.argmax(weights)
    max_weight = weights[max_weight_index]
    print(usernames[index], round(max_weight*100,2), booktitles[books[max_weight_index]])
print(booktitles[books[np.argmax(fav_summary)]])

aazadh 1.01 Meerayude Novellakal
abdul rahoof s 1.09 Python Essential Reference
abhinand c. s. 0.31 Bioinformatics: Sequence and Genome Analysis
abhinaya 0.54 Introduction to Bioinformatics
ajay pradeep 1.4 MYSQL 5.0
akhil vasim 4.34 Introduction to Bioinformatics
akhila s 1.71 Basic Bioinformatics
akhiya a.r. 2.87 Mastering Perl for Bioinformatics
alan syed 1.01 Database System Concepts
amritha raju 0.39 Bioinformatics - A Practical guide to the Analysis of Gene
amrutha p 1.4 SQL, PL /SQL: The Programming Language of Oracle
anagha rajan 3.1 UGC CSIR-JRF/ NET Compulsory   Paper - 1: Common for science subjects
anas a s 1.24 Cell Biology
ann 0.39 World' Most Humorous Short Stories
anu sasi 1.09 Research methodology: Methods and techniques
anuja mohan 0.78 Biotechnology For B.sc
anuroopa g nadh 6.74 Dravya Guna Vijnana : Study of the Essential Medicinal Plants in Ayurveda Vol.2 
arjun v.m. 0.23 Databases
arun k unni 0.54 Secrets of Success: The science and spirit of real prosperity
arun 

# Machine Learning Based Prediction

In [35]:
input_data = []; output_labels = []
for user in usage_record:
    for i in range(len(user)):
        subset = user[i:i+RANGE]
        if len(subset)==RANGE:
            input_data.append(subset[0:-1])
            output_labels.append(subset[-1])
X=np.array(input_data); Y=np.array(output_labels)

In [40]:
model = Sequential([
    Embedding(max_embed, 256, input_length=4),
    LSTM(256),
    Dense(max_embed, activation='softmax')])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [41]:
model.fit(X, Y, batch_size=100, epochs=10)
loss = model.evaluate(X, Y, verbose=0)

Epoch 1/10
10/10 [==============================] - 3s 53ms/step - loss: 6.8776
Epoch 2/10
10/10 [==============================] - 1s 51ms/step - loss: 6.8571
Epoch 3/10
10/10 [==============================] - 1s 50ms/step - loss: 6.8342
Epoch 4/10
10/10 [==============================] - 1s 50ms/step - loss: 6.7890
Epoch 5/10
10/10 [==============================] - 1s 52ms/step - loss: 6.6454
Epoch 6/10
10/10 [==============================] - 1s 72ms/step - loss: 6.4735
Epoch 7/10
10/10 [==============================] - 1s 77ms/step - loss: 6.2542
Epoch 8/10
10/10 [==============================] - 1s 81ms/step - loss: 6.0217
Epoch 9/10
10/10 [==============================] - 1s 80ms/step - loss: 5.6842
Epoch 10/10
10/10 [==============================] - 1s 78ms/step - loss: 5.1304


In [43]:
test_books = ["DCB3311","DCB2992","DCB2789","DCB3326"]
seed = [books.index(i) for i in test_books]
print(predict_book(seed, model, books, booktitles))

(674, 'DCB3858', 'Machine learning with Python cookbook :')
